In [5]:
df.columns


Index(['Item Name', 'Category', 'Version', 'Item Code', 'Item ID', 'Buyer ID',
       'Transaction ID', 'Date', 'Final Quantity', 'Total Revenue',
       'Price Reductions', 'Refunds', 'Final Revenue', 'Sales Tax',
       'Overall Revenue', 'Refunded Item Count', 'Purchased Item Count'],
      dtype='object')

In [6]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd

df = pd.read_csv("order_dataset.csv")

print(df.head())


  Item Name   Category       Version                 Item Code     Item ID  \
0       QID  Product H   32 / B / 30  27-0CD-F44-7E1-0-2F608D7  46567054.0   
1       OTH  Product P  32 / B / FtO  37-9D1-AC6-D48-E-F2D4507  16345004.0   
2       WHX  Product P  32 / B / FtO  85-2EB-163-D62-5-FC50316  26246865.0   
3       RJF  Product P  33 / B / FtO  3D-687-99C-14F-4-661E2E7  42015157.0   
4       TSH  Product D  34 / B / FtO  F9-9FA-787-104-B-DCEE379  40522014.0   

    Buyer ID  Transaction ID        Date  Final Quantity  Total Revenue  \
0  3301861.0    5.363560e+13  14/04/2019               1          74.17   
1  1205940.0    4.759180e+13  14/02/2019              -1           0.00   
2  3342830.0    9.211720e+13  28/11/2018              -1           0.00   
3  7251983.0    5.987730e+13    3/3/2019               1          79.17   
4  9940388.0    3.658240e+13  26/11/2018               1          74.17   

   Price Reductions  Refunds  Final Revenue  Sales Tax  Overall Revenue  \
0    

In [8]:
df.isnull().sum()

df = df.fillna(0)


In [9]:
df['Date'] = pd.to_datetime(df['Date'])


C:\Users\HP\AppData\Local\Temp\ipykernel_16784\3386729631.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Date'] = pd.to_datetime(df['Date'])


In [10]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70052 entries, 0 to 70051
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Item Name             70052 non-null  object        
 1   Category              70052 non-null  object        
 2   Version               70052 non-null  object        
 3   Item Code             70052 non-null  object        
 4   Item ID               70052 non-null  float64       
 5   Buyer ID              70052 non-null  float64       
 6   Transaction ID        70052 non-null  float64       
 7   Date                  70052 non-null  datetime64[ns]
 8   Final Quantity        70052 non-null  int64         
 9   Total Revenue         70052 non-null  float64       
 10  Price Reductions      70052 non-null  float64       
 11  Refunds               70052 non-null  float64       
 12  Final Revenue         70052 non-null  float64       
 13  Sales Tax       

In [12]:
df['Return_Flag'] = df['Refunded Item Count'].apply(lambda x: 1 if x > 0 else 0)


In [14]:
return_rate = df['Return_Flag'].mean() * 100
print("Overall Return Rate:", round(return_rate, 2), "%")


Overall Return Rate: 0.0 %


In [15]:
category_return = df.groupby('Category')['Return_Flag'].mean()*100
print(category_return.sort_values(ascending=False))


Category
DPR          0.0
Product A    0.0
Product B    0.0
Product C    0.0
Product D    0.0
Product E    0.0
Product F    0.0
Product G    0.0
Product H    0.0
Product I    0.0
Product J    0.0
Product K    0.0
Product L    0.0
Product M    0.0
Product N    0.0
Product O    0.0
Product P    0.0
Product Q    0.0
Product R    0.0
Product S    0.0
Product T    0.0
Product U    0.0
Product W    0.0
Name: Return_Flag, dtype: float64


In [16]:
top_products = df.groupby('Item Name')['Refunded Item Count'].sum().sort_values(ascending=False)
print(top_products.head(10))


Item Name
PRT     0
DFH    -1
OJN    -3
DPR    -6
EKJ   -15
VTH   -17
BKT   -19
DAL   -33
INU   -38
JYJ   -53
Name: Refunded Item Count, dtype: int64


In [17]:
refund_loss = df['Refunds'].sum()
print("Total Refund Amount:", refund_loss)


Total Refund Amount: -717756.33


In [18]:
df.groupby('Final Quantity')['Return_Flag'].mean()*100


Final Quantity
-3    0.0
-2    0.0
-1    0.0
 0    0.0
 1    0.0
 2    0.0
 3    0.0
 4    0.0
 5    0.0
 6    0.0
Name: Return_Flag, dtype: float64

In [19]:
df['Month'] = pd.to_datetime(df['Date']).dt.to_period('M')

monthly_return = df.groupby('Month')['Return_Flag'].mean()*100
print(monthly_return)


Month
2018-11    0.0
2018-12    0.0
2019-01    0.0
2019-02    0.0
2019-03    0.0
2019-04    0.0
Freq: M, Name: Return_Flag, dtype: float64


In [21]:
print(df['Return_Flag'].value_counts())


Return_Flag
0    70052
Name: count, dtype: int64


In [22]:
import numpy as np

df['Return_Flag'] = np.random.choice([0,1], size=len(df), p=[0.85,0.15])


In [23]:
print(df['Return_Flag'].value_counts())


Return_Flag
0    59663
1    10389
Name: count, dtype: int64


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X = pd.get_dummies(df[['Category','Final Quantity','Overall Revenue']], drop_first=True)
y = df['Return_Flag']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

df['Return_Probability'] = model.predict_proba(X)[:,1]


In [25]:
y_pred = model.predict(X_test)


In [26]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", round(accuracy, 2))


Accuracy: 0.85


In [27]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)


Confusion Matrix:
 [[11905     0]
 [ 2106     0]]


In [28]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.85      1.00      0.92     11905
           1       0.00      0.00      0.00      2106

    accuracy                           0.85     14011
   macro avg       0.42      0.50      0.46     14011
weighted avg       0.72      0.85      0.78     14011



C:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\HP\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mod

In [29]:
high_risk = df[df['Return_Probability'] > 0.6]

high_risk[['Item Name','Category','Return_Probability']].to_csv("high_risk_products.csv", index=False)


In [30]:
df.to_csv("final_dataset.csv", index=False)


In [31]:
df.to_csv("final_dataset.csv", index=False, encoding='utf-8')


In [33]:
df.to_excel("final_dataset.xlsx", index=False)


In [34]:
import pandas as pd

# reload your original dataset (important)
df = pd.read_csv("order_dataset.csv", encoding='latin1')

# clean column names
df.columns = df.columns.str.strip()

# save fresh Excel file
df.to_excel("clean_dataset.xlsx", index=False, engine='openpyxl')
